In [ ]:
!pip install pinecone

In [ ]:
!pip3 install sentence-transformers

In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/medium_data.csv

--2025-11-14 10:05:57--  https://s3-geospatial.s3.us-west-2.amazonaws.com/medium_data.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.249.218, 3.5.83.150, 52.92.195.210, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.249.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 599580 (586K) [text/csv]
Saving to: ‘medium_data.csv’

medium_data.csv     100%[===================>] 585.53K  --.-KB/s    in 0.1s    

2025-11-14 10:05:57 (3.83 MB/s) - ‘medium_data.csv’ saved [599580/599580]



In [ ]:
!wc -l medium_data.csv

2499 medium_data.csv


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("medium_data.csv") # excercise whole data set

In [ ]:
df.head()

,id,url,title,subtitle,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",NaN,188.0,0,24,Towards Data Science,27-01-2023
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,NaN,10.0,0,13,Towards Data Science,27-01-2023


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2498 entries, 0 to 2497
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2498 non-null   int64  
 1   url           2498 non-null   object 
 2   title         2498 non-null   object 
 3   subtitle      2073 non-null   object 
 4   claps         2423 non-null   float64
 5   responses     2498 non-null   int64  
 6   reading_time  2498 non-null   int64  
 7   publication   2498 non-null   object 
 8   date          2498 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 175.8+ KB


In [ ]:
df['title'] = df['title'].astype(str).fillna('')
df['subtitle'] = df['subtitle'].astype(str).fillna('')

In [ ]:
df['metadata'] = df.apply(lambda row: {'title': row['title'] + " " + row['subtitle']}, axis=1)

In [ ]:
df.head()

,id,url,title,subtitle,claps,responses,reading_time,publication,date,metadata
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023,{'title': 'Not All Rainbows and Sunshine: The ...
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023,{'title': 'Ethics in AI: Potential Root Causes...
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",nan,188.0,0,24,Towards Data Science,27-01-2023,"{'title': 'Python Tuple, The Whole Truth and O..."
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023,{'title': 'Dates and Subqueries in SQL Working...
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,nan,10.0,0,13,Towards Data Science,27-01-2023,{'title': 'Temporal Differences with Python: F...


In [ ]:
from google.colab import userdata

# Use Colab Secrets
API_KEY = userdata.get('pinecone_api_key')

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=API_KEY)

In [ ]:
from pinecone import ServerlessSpec

# Spec for free tier plan
spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)


In [ ]:
index_name = 'semantic-search-fast'

In [ ]:
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

In [ ]:
existing_indexes

['semantic-search-fast']

In [ ]:
import time

# If the following code runs more than once, you might get "The index already existed" error
# In that case, delete the index first: pc.delete_index(index_name)
try:
    pc.delete_index(index_name)
except Exception as e:
    print(e)

pc.create_index(
    index_name,
    dimension=384,  # dimensionality of minilm
    metric='dotproduct',
    spec=spec
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [ ]:
#Generate sentence embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device='cpu')
df['values'] = df['metadata'].apply(lambda x: model.encode(x['title']).tolist())

In [ ]:
#Creating a Dataframe for Index Ingestion
df_upsert = df[['id', 'values', 'metadata']]
df_upsert['id'] = df_upsert['id'].astype(str)

/tmp/ipython-input-2904977951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upsert['id'] = df_upsert['id'].astype(str)


In [ ]:
#Ingesting the Dataframe into Pinecone
index = pc.Index(index_name)
index.upsert_from_dataframe(df_upsert)

sending upsert requests:   0%|          | 0/2498 [00:00<?, ?it/s]

{'upserted_count': 2498}

In [ ]:
#Run search against Pinecone
xc = index.query(
vector=(model.encode("what is ethics in AI")).tolist(), # python list
top_k=10,
include_metadata=True,
include_values=True
)

In [ ]:
for result in xc['matches']:
    print(result['id'], result['score'], result['metadata'])

1635 0.740670681 {'title': 'Ethics in AI: Potential Root Causes for Biased Algorithms An alternative approach to understanding bias in\xa0data'}
2 0.740670681 {'title': 'Ethics in AI: Potential Root Causes for Biased Algorithms An alternative approach to understanding bias in\xa0data'}
1327 0.716887474 {'title': 'The ethical implications of AI in\xa0design It’s time to talk ethics, chatGPT, Midjourney &\xa0more.'}
662 0.667053223 {'title': 'Ethical Considerations In Machine Learning\xa0Projects Don’t forget these topics when building AI\xa0systems'}
1628 0.651916 {'title': 'Navigating the Ethical Contours of AI Copy Generators Like\xa0ChatGPT Be prepared to cite the\xa0AI…'}
2030 0.651916 {'title': 'Navigating the Ethical Contours of AI Copy Generators Like\xa0ChatGPT Be prepared to cite the\xa0AI…'}
1345 0.607173443 {'title': 'Using AI in UX research: a structured and ethical\xa0approach A structured approach for assessing the…'}
370 0.596168041 {'title': 'The Path towards AI Regulati